In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
# Loading dataset containing first five categories
train = pd.read_csv('../self_training/train.csv')
test = pd.read_csv('../self_training/test.csv')
dados = pd.concat([train,test])
dados.head(20)

,indice,num_processos,classe_judicial,tipo,rotulo_nome,rotulo_num,texto
0,0,0017455-09.2000.8.07.0001\n,EXECUÇÃO FISCAL (1116)\n,Certidão,NaN,NaN,EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: AUTAMI...
1,1,0017455-09.2000.8.07.0001\n,EXECUÇÃO FISCAL (1116)\n,Despacho,NaN,NaN,EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: AUTAMI...
2,3,0017455-09.2000.8.07.0001\n,EXECUÇÃO FISCAL (1116)\n,Certidão,NaN,NaN,EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: AUTAMI...
3,4,0025910-26.2001.8.07.0001\n,EXECUÇÃO FISCAL (1116)\n,Certidão,NaN,NaN,APELANTE: DISTRITO FEDERAL\nAPELADO: ENCOL S/A...
4,6,0026055-06.2016.8.07.0018\n,EMBARGOS À EXECUÇÃO FISCAL (1118)\n,Despacho,NaN,NaN,EXEQUENTE: CARREFOUR COMERCIO E INDUSTRIA LTDA...
5,7,0026055-06.2016.8.07.0018\n,EMBARGOS À EXECUÇÃO FISCAL (1118)\n,Certidão,NaN,NaN,EXEQUENTE: CARREFOUR COMERCIO E INDUSTRIA LTDA...
6,9,0026055-06.2016.8.07.0018\n,EMBARGOS À EXECUÇÃO FISCAL (1118)\n,Despacho,NaN,NaN,EXEQUENTE: CARREFOUR COMERCIO E INDUSTRIA LTDA...
7,17,0026471-85.2013.8.07.0015\n,UMPRIMENTO DE SENTENÇA (156)\n,Certidão,NaN,NaN,EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: ALFA A...
8,18,0026540-38.2008.8.07.0001\n,EXECUÇÃO FISCAL (1116)\n,Despacho,NaN,NaN,EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: JAIRA ...
9,20,0026568-45.2004.8.07.0001\n,EXECUÇÃO FISCAL (1116)\n,Decisão,Art40_Penhora,3.0,EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: VENEZA...


In [3]:
# Parameters
sdg_params = dict(alpha=1e-5, penalty="l2", loss="log")
vectorizer_params = dict(ngram_range=(1, 2), min_df=5, max_df=0.8)

In [4]:
# Supervised Pipeline
pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        ("clf", SGDClassifier(**sdg_params)),
    ]
)
# LabelSpreading Pipeline
ls_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(**vectorizer_params)),
        ("tfidf", TfidfTransformer()),
        # LabelSpreading does not support dense matrices
        ("todense", FunctionTransformer(lambda x: x.todense())),
        ("clf", LabelSpreading()),
    ]
)

In [5]:
def eval_and_print_metrics(clf, X_train, y_train, X_test, y_test):
    print("Número de amostras de treinamento:", len(X_train))
    print("Amostras não rotuladas no conjunto de treinamento:", sum(1 for x in y_train if x == -1))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(
        "Micro-averaged F1 score on test set: %0.3f"
        % f1_score(y_test, y_pred, average="micro")
    )
    print("-" * 10)
    print()
    
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

In [6]:
X, y = dados.texto, dados.rotulo_num
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
X_test

577     EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: POLY P...
563     Classe judicial: EXECUÇÃO FISCAL (1116)\nEXEQU...
3307    Classe judicial: EXECUÇÃO FISCAL (1116)\nEXEQU...
1410    EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: ESPOLI...
2369    EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: ESPOLI...
                              ...                        
2940    EXEQUENTE: DISTRITO FEDERAL\nESPÓLIO DE: NADIR...
1048    EMBARGANTE: BANCO ITAUCARD S.A.\nEMBARGADO: DI...
989     EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: JOSE G...
3095    DE SABRINA DAMASCENO VIANA NUNES. .\nFoi deter...
1618    EXEQUENTE: DISTRITO FEDERAL\nEXECUTADO: ESPOLI...
Name: texto, Length: 993, dtype: object

In [9]:
y_test

577    NaN
563    NaN
3307   NaN
1410   NaN
2369   NaN
        ..
2940   NaN
1048   NaN
989    NaN
3095   NaN
1618   NaN
Name: rotulo_num, Length: 993, dtype: float64

In [ ]:
print("SGDClassifier supervisionado em 100% dos dados:")
eval_and_print_metrics(pipeline, X_train, y_train, X_test, y_test)

# selecione uma máscara de 20% do conjunto de dados do treino
y_mask = np.random.rand(len(y_train)) < 0.0216

# X_20 e y_20 são o subconjunto do conjunto de dados do treino indicado pela máscara
X_20, y_20 = map(
    list, zip(*((x, y) for x, y, m in zip(X_train, y_train, y_mask) if m))
)
print("SGDClassifier supervisionado em 20% dos dados de treinamento:")
eval_and_print_metrics(pipeline, X_20, y_20, X_test, y_test)

# definir o subconjunto não mascarado para não rotulado
y_train[~y_mask] = -1

print("LabelSpreading em 20% dos dados (o restante não foi rotulado):")
eval_and_print_metrics(ls_pipeline, X_train, y_train, X_test, y_test)